# LNS Example 1: Knapsack

The first example is the knapsack problem. We have a set of items $I$ with a weight $w_i$ and a value $v_i$.
We want to select a subset of items such that the total weight does not exceed a given capacity $C$ and the total value is maximized.

$$\max \sum_{i \in I} v_i x_i$$
$$\text{s.t.} \sum_{i \in I} w_i x_i \leq C$$
$$x_i \in \{0,1\}$$

This is one of the simplest NP-hard problems and can be solved with a dynamic programming approach in pseudo-polynomial time.
CP-SAT is also able to solve many large instances of this problem in an instant.
However, its simple structure makes it a good example to demonstrate the use of Large Neighborhood Search, even if the algorithm will
not be of much use for this problem.

In [17]:
# import all dependencies
import typing
import math
import random
from collections import namedtuple

from ortools.sat.python import cp_model

## Instance Generation

First, we need to create some random instances.

In [18]:
# Instance generation
class Item:
    """
    A simple class to represent an item in the knapsack problem.
    Every instance of this class is unique, i.e., two items with
    the same weight and value are not equal. Otherwise, we could
    only have a single item for each weight and value combination.
    """

    def __init__(self, weight: int, value: int):
        self.weight = weight
        self.value = value

    def __repr__(self):
        return f"Item({self.weight}, {self.value})"

    def __eq__(self, other):
        return id(self) == id(other)

    def __hash__(self):
        return id(self)


class Instance:
    """
    Simple instance container.
    """

    def __init__(self, items: typing.List[Item], capacity: int) -> None:
        self.items = items
        self.capacity = capacity
        assert len(items) > 0
        assert capacity > 0


def random_instance(num_items: int, ratio: float) -> Instance:
    """
    Creates a random instance of the knapsack problem.
    :param num_items: The number of items.
    :param ratio: The ratio between capacity and sum of weights.
    :return: A list of items and a capacity
    """
    items = []
    for i in range(num_items):
        weight = random.randint(10, 1000)
        value = round(random.triangular(1, 100, 5) * weight)
        items.append(Item(weight, value))
    capacity = math.ceil(sum(item.weight for item in items) * ratio)
    return Instance(items, capacity)


def value(items: typing.List[Item]) -> int:
    """
    Returns the total value of a list of items.
    """
    return sum(item.value for item in items)

## Greedy Algorithm

Next, we need an initial solution.
We use a simple greedy algorithm that adds items to the knapsack as long as the capacity is not exceeded.
It would be much smarter to sort the items by value/weight ratio and add the items with the highest ratio first.
However, this would often create near-optimal solution, and then we wouldn't see much improvement from the LNS.

In [19]:
# Simple greedy algorithm for the knapsack problem
def greedy_solution(instance: Instance) -> typing.List[Item]:
    """
    A simple greedy algorithm for the knapsack problem.
    It is bad on purpose, so we can improve it with local search.
    For random instances, the greedy algorithm otherwise often
    finds the (nearly) optimal solution and there is nothing to see.
    """
    solution = []
    remaining_capacity = instance.capacity
    for item in instance.items:
        if item.weight <= remaining_capacity:
            solution.append(item)
            remaining_capacity -= item.weight
    return solution

## Exact Solver for Subproblem

We will remove items from the knapsack and try to refill it with better items.
This subproblem is the Knapsack problem again, and we can solve it with CP-SAT.

In [20]:
# Exact solver for knapsack


def solve_knapsack(
    instance: Instance, max_time_in_seconds: float = 90
) -> typing.List[Item]:
    """
    Optimal solver for knapsack
    """
    model = cp_model.CpModel()
    x = [model.new_bool_var(f"x_{i}") for i in range(len(instance.items))]
    model.add(
        sum(x[i] * item.weight for i, item in enumerate(instance.items))
        <= instance.capacity
    )
    model.maximize(sum(x[i] * item.value for i, item in enumerate(instance.items)))
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = max_time_in_seconds
    # solver.parameters.log_search_progress = True
    status = solver.solve(model)
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        if status == cp_model.FEASIBLE:
            print(
                "Warning: Solver did not find optimal solution. Returned solution is feasible but not optimal."
            )
        return [
            item for i, item in enumerate(instance.items) if solver.value(x[i]) == 1
        ]
    else:
        return []

## Large Neighborhood Search for the Knapsack Problem

### Initialization

We need to start with an initial solution that is then improved by the LNS algorithm.
We use a simple greedy algorithm to generate an initial solution.

In [21]:
# Create instance
instance = random_instance(1_000_000, 0.1)
# compute some initial solution
initial_solution = greedy_solution(instance)

### Improve the solution via LNS

The LNS algorithm is a heuristic that iteratively destroys and repairs parts of the solution.
We remove a part of the selected item in the current solution and then select some additional
items from the remaining set. Using the exact solver, we find the optimal solution for the
remaining capacity using the selected items. This is repeated for some iterations.

There are two important parameters for the LNS algorithm:
* The size of the subproblem we solve with the exact solver.
* The size of items we remove from the current solution.

In [24]:
class KnapsackLns:
    """
    Knapsack LNS solver.
    """

    def __init__(
        self,
        instance: Instance,
        initial_solution: typing.List[Item],
        subproblem_size: int,
    ):
        self.instance = instance
        self.solution = initial_solution
        self.subproblem_size = (
            subproblem_size  # Number of items to consider in subproblem
        )
        self.solutions = [initial_solution]

    def _remaining_capacity(self):
        return self.instance.capacity - sum(item.weight for item in self.solution)

    def _remaining_items(self):
        return list(set(self.instance.items).difference(self.solution))

    def _destroy(self, num_items: int) -> typing.List[Item]:
        """
        Destroy a part of the solution by removing num_items from it.
        """
        num_items = min(len(self.solution), num_items)
        assert 0 <= num_items <= self.subproblem_size
        items_removed = random.sample(self.solution, num_items)
        self.solution = [item for item in self.solution if item not in items_removed]
        print(
            f"Removed {len(items_removed)} items from solution. New remaining capacity: {self._remaining_capacity()}"
        )
        return items_removed

    def _repair(self, I_: typing.List[Item], max_time_in_seconds: float = 90):
        """
        Repair the solution by adding items from I_ to it.
        """
        C_ = self._remaining_capacity()
        print(
            f"Repairing solution by considering {len(I_)} items to fill the remaining capacity of {C_}."
        )
        subsolution = solve_knapsack(Instance(I_, C_), max_time_in_seconds)
        self.solution += subsolution
        assert self._remaining_capacity() >= 0

    def perform_lns_iteration(
        self, destruction_size: int, max_time_in_seconds: float = 90
    ):
        # 1. Destroy
        assert destruction_size > 0
        items_removed = self._destroy(destruction_size)
        # 2. Build subproblem for repair
        remaining_items = self._remaining_items()
        n = min(self.subproblem_size - destruction_size, len(remaining_items))
        new_items_to_consider = random.sample(remaining_items, n)
        # Add the removed items to the set of items to consider, such that
        # we can also find an equally good solution
        I_ = list(set(items_removed + new_items_to_consider).difference(self.solution))
        # 3. Repair
        self._repair(I_, max_time_in_seconds)
        self.solutions.append(self.solution)

### Run the LNS

Play around with the parameters and see how the LNS algorithm improves the solution.

In [27]:
lns = KnapsackLns(instance, initial_solution, subproblem_size=10_000)
for i in range(25):
    lns.perform_lns_iteration(destruction_size=1_000)
    print(
        f"=> Iteration {i}: {value(lns.solution)} (improvement: {value(lns.solution) / value(lns.solutions[0])})"
    )

Removed 1000 items from solution. New remaining capacity: 499228
Repairing solution by considering 9993 items to fill the remaining capacity of 499228.
iteration 0: 1805895706 (improvement: 1.01254478180439)
Removed 1000 items from solution. New remaining capacity: 498662
Repairing solution by considering 9981 items to fill the remaining capacity of 498662.
iteration 1: 1827800672 (improvement: 1.0248266422380858)
Removed 1000 items from solution. New remaining capacity: 496984
Repairing solution by considering 9985 items to fill the remaining capacity of 496984.
iteration 2: 1849866865 (improvement: 1.0371989007811484)
Removed 1000 items from solution. New remaining capacity: 510463
Repairing solution by considering 9993 items to fill the remaining capacity of 510463.
iteration 3: 1871933620 (improvement: 1.0495714744310944)
Removed 1000 items from solution. New remaining capacity: 499582
Repairing solution by considering 9987 items to fill the remaining capacity of 499582.
iteration 

## Try to compute an optimal solution

To have a comparison, we can try to compute an optimal solution with the exact solver.

In [26]:
optimal_solution = solve_knapsack(instance, max_time_in_seconds=900)
print(f"CP-SAT solution: {value(optimal_solution)}")

Optimal solution: 1800396315


## Conclusion

We are able to improve the initial solution via LNS, however, only because we used a bad greedy algorithm.
If we had used a better greedy algorithm, the LNS algorithm would not be able to improve the solution by much.
However, the LNS algorithm is a very powerful heuristic that can be used to improve solutions for many problems.
This example just had the purpose to demonstrate the implementation of LNS.

## Exercise

Try to generalize the LNS algorithm for Multi-Knapsack problems, where instead of a single knapsack, we have multiple knapsacks with different capacities, and items can have different values and weights for each knapsack.
Multi-Knapsack problems can be significantly harder but also of practical interest for many applications, such as scheduling and resource allocation.